In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress
from urllib.parse import urlparse
from tldextract import extract
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils.dataframe import column as column_utils


from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report

import lightgbm as lgb

np.set_printoptions(threshold=np.nan)
import pickle



plt.rcParams["figure.figsize"] = (24,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import missingno as msno
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
import datetime
from scipy import signal
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn import linear_model
from sklearn.metrics import roc_auc_score



InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_seq_items', None)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')
import dask.dataframe as dd

In [3]:
def read_cv():
    return pd.read_hdf("data/cv2_2.h5","table")

def read_train():
    df_train = pd.read_hdf("data/cv1_2.h5","table")
#     df_pos = df_train[df_train['is_attributed']==1]
#     df_negs = df_train[df_train['is_attributed']==0]
#     df_negs = df_negs.sample(frac=0.1)
#     df_train = pd.concat([df_pos,df_negs],ignore_index=True)
    return df_train

def save_model(model,filename):
    pickle.dump(model, open(filename, "wb"))
    
def load_model(filename):
    return pickle.load(open("xgc.pickle.dat", "rb"))

def train_xgc_incrementally(xgc,training_data_getter,cv_getter,features=None,feature_getter=None,target='target',splits=[0.25,0.25,0.25,0.25]):
    from sklearn.metrics import log_loss
    gc.collect()
    if(abs(np.sum(splits)-1)>1e-9):
        raise ValueError("sum of splits should be 1")
    df = training_data_getter()
    if(features is None):
        features = feature_getter(df.head())
    
    length = df.shape[0]
    gc.collect()
    # df = None
    # del df
    probs = np.random.rand(length)
    
    lower = 0
    upper = 0
    old_X = None
    old_y = None
    results = None
    df_cv = cv_getter()
    X_cv = df_cv[features]
    y_cv = df_cv[target]
    df_cv = None
    del df_cv
    for i in splits:
        
        upper = lower+i
        # print("Training Split, Lower: %s, Upper: %s"%(lower,upper))
        gc.collect()
        # df = training_data_getter()
        
        
        mask = (probs>=lower) & (probs < upper)
        df_new = df[mask]
        # df = None
        # del df
        
        X = df_new[features]
        y = df_new[target]
        
        xgc.fit(X,y)
        if(old_X is None and old_y is None):
            old_X = X
            old_y = y

        gc.collect()
        y_scores = xgc.predict_proba(old_X)[:,1]
        auc = roc_auc_score(old_y, y_scores)
        log_l = log_loss(old_y, y_scores)
        y_scores = xgc.predict(old_X)
        # cm = model_utils.confusion_matrix_frame(old_y,y_scores)
        # print("Auc = %s, Log Loss = %s, Confusion Matrix = "%(auc,log_l))
        # display(cm)
        
        y_scores_cv = xgc.predict_proba(X_cv)[:,1]
        auc_cv = roc_auc_score(y_cv, y_scores_cv)
        log_l_cv = log_loss(y_cv, y_scores_cv)
        
        print("Auc Train = %.5f, Log Loss Train = %.5f, Auc test= %.5f, Log Loss Test= %.5f"%(auc,log_l,auc_cv,log_l_cv))
        
        res = np.array([[auc,log_l,auc_cv,log_l_cv]])
        if(results is None):
            results = res
        else:
            results = np.append(results,res,axis=0)
        
        
            
        
        df_new = None
        del df_new
        
        old_X = X
        old_y = y
        X = None
        y = None
        del X
        del y
        
        
        lower = upper
    gc.collect()
    results = pd.DataFrame(results,columns=["auc_train","Log loss train","auc_test","Log loss test"])
    display(results)
    return xgc,features,df.head()

In [4]:

def feature_getter(df):
    return list(set(df.columns) - set(['attributed_time','click_time','is_attributed']))
xgc = XGBClassifier(max_depth=8, learning_rate=0.6,n_estimators=100,n_jobs=46,missing=np.nan,scale_pos_weight=200,gamma=2,eval_metric="auc")

In [5]:
xgc,features,sample_df = train_xgc_incrementally(xgc,
                              read_train,
                              cv_getter=read_cv,
                              feature_getter=feature_getter,
                              target='is_attributed',
                              splits=[0.25,0.25,0.25,0.25])

Auc Train = 0.99714, Log Loss Train = nan, Auc test= 0.96536, Log Loss Test= nan
Auc Train = 0.96599, Log Loss Train = nan, Auc test= 0.96556, Log Loss Test= nan
Auc Train = 0.96523, Log Loss Train = nan, Auc test= 0.96553, Log Loss Test= nan
Auc Train = 0.96662, Log Loss Train = nan, Auc test= 0.96642, Log Loss Test= nan


,auc_train,Log loss train,auc_test,Log loss test
0,0.997141,NaN,0.965363,NaN
1,0.965992,NaN,0.965558,NaN
2,0.965227,NaN,0.965530,NaN
3,0.966621,NaN,0.966417,NaN


In [4]:
xgc,features,sample_df = train_xgc_incrementally(xgc,
                              read_train,
                              cv_getter=read_cv,
                              feature_getter=feature_getter,
                              target='is_attributed',
                              splits=[0.2,0.2,0.2,0.2,0.2])

Auc Train = 0.97445, Log Loss Train = 0.06369, Auc test= 0.97975, Log Loss Test= 0.00902
Auc Train = 0.97361, Log Loss Train = 0.06465, Auc test= 0.97993, Log Loss Test= 0.00896
Auc Train = 0.97371, Log Loss Train = 0.06471, Auc test= 0.98201, Log Loss Test= 0.00898
Auc Train = 0.97269, Log Loss Train = 0.06475, Auc test= 0.97709, Log Loss Test= 0.00921
Auc Train = 0.97399, Log Loss Train = 0.06371, Auc test= 0.98240, Log Loss Test= 0.00899


,auc_train,Log loss train,auc_test,Log loss test
0,0.974453,0.063688,0.979750,0.009019
1,0.973606,0.064652,0.979927,0.008963
2,0.973706,0.064714,0.982013,0.008983
3,0.972687,0.064750,0.977086,0.009213
4,0.973993,0.063707,0.982396,0.008991


In [4]:
xgc,features,sample_df = train_xgc_incrementally(xgc,
                              read_train,
                              cv_getter=read_cv,
                              feature_getter=feature_getter,
                              target='is_attributed',
                              splits=[0.2,0.2,0.2,0.2,0.2])

Auc Train = 0.97340, Log Loss Train = 0.06436, Auc test= 0.97703, Log Loss Test= 0.00906
Auc Train = 0.97285, Log Loss Train = 0.06398, Auc test= 0.97867, Log Loss Test= 0.00899
Auc Train = 0.97127, Log Loss Train = 0.06749, Auc test= 0.97632, Log Loss Test= 0.00892
Auc Train = 0.97176, Log Loss Train = 0.06499, Auc test= 0.98297, Log Loss Test= 0.00866
Auc Train = 0.97153, Log Loss Train = 0.06627, Auc test= 0.97867, Log Loss Test= 0.00889
Auc Train = 0.97299, Log Loss Train = 0.06668, Auc test= 0.97677, Log Loss Test= 0.00910
Auc Train = 0.97342, Log Loss Train = 0.06498, Auc test= 0.97806, Log Loss Test= 0.00891
Auc Train = 0.97315, Log Loss Train = 0.06465, Auc test= 0.97925, Log Loss Test= 0.00907
Auc Train = 0.97253, Log Loss Train = 0.06598, Auc test= 0.97888, Log Loss Test= 0.00897
Auc Train = 0.97316, Log Loss Train = 0.06416, Auc test= 0.97985, Log Loss Test= 0.00885


,auc_train,Log loss train,auc_test,Log loss test
0,0.973396,0.064355,0.977028,0.009065
1,0.972853,0.063978,0.978673,0.008993
2,0.971267,0.067487,0.976322,0.008918
3,0.971762,0.064991,0.982969,0.008662
4,0.971528,0.066272,0.978670,0.008892
5,0.972991,0.066677,0.976765,0.009103
6,0.973415,0.064979,0.978057,0.008914
7,0.973148,0.064655,0.979250,0.009068
8,0.972526,0.065976,0.978878,0.008966
9,0.973162,0.064160,0.979850,0.008845


In [ ]:


# df_train = read_cv()
df_train = read_train()
# df_cv2 = pd.read_hdf("data/cv2_2.h5","table")

gc.collect()




In [ ]:
df_utils.get_column_names(df_train)

In [ ]:
def check_feature_usefulness(df,feature):
    means = df_cv1.groupby(['is_attributed'])[[feature]].mean()
    print("Mean values =")
    display(means)
    
    d0 = df_cv1[df_cv1["is_attributed"]==0][[feature]].describe()
    d1 = df_cv1[df_cv1["is_attributed"]==1][[feature]].describe()
    print("Describe 0 =")
    display(d0)
    print("Describe 1 =")
    display(d1)
    
    max_0 = df_cv1[df_cv1["is_attributed"]==0][feature].max()
    max_1 = df_cv1[df_cv1["is_attributed"]==1][feature].max()
    print("Max for 0= %s, Max for 1= %s" %(max_0,max_1))
    
    max_0_samples = df[df[feature]>(max_0-1e-6)].sample(10)
    max_1_samples = df[df[feature]>(max_1-1e-6)].sample(10)
    
    print("Samples from is_attributed=0 and is_attributed=1 with max values")
    display(max_0_samples)
    display(max_1_samples)
    df = df.sample(frac=0.01)
    sns.stripplot(x="is_attributed", y=feature, data=df,jitter=True);
    plt.show()
    sns.violinplot(x="is_attributed", y=feature, data=df);
    plt.show()

check_feature_usefulness(df_train,'os_hour_count_channel')

In [4]:
df_pos = df_train[df_train['is_attributed']==1]
df_negs = df_train[df_train['is_attributed']==0]
df_negs = df_negs.sample(frac=0.7)
df_train = pd.concat([df_pos,df_negs],ignore_index=True)
df_pos = None
df_negs = None
gc.collect()

0

In [ ]:
features = list(set(df_train.columns) - set(['attributed_time','click_time','is_attributed']))
output_column = 'is_attributed'

# features = ['os','ip','app','channel','hour','device']
# features.extend(['attributed_time','click_time','is_attributed'])
# features = list(set(df_train.columns) - set(features))

In [6]:
df_train = df_train.sample(frac=0.8)

gc.collect()
X = df_train[features]
y = df_train[output_column]
gc.collect()
df_train = None
del df_train
gc.collect()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=17)


In [7]:
gc.collect()
xgc = XGBClassifier(max_depth=8, learning_rate=0.7,n_estimators=200,n_jobs=46,missing=np.nan,scale_pos_weight=0.07,gamma=4,eval_metric="auc")
xgc.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=4, learning_rate=0.7,
       max_delta_step=0, max_depth=8, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=46, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=0.07, seed=None, silent=True,
       subsample=1)

In [17]:
df_i = model_utils.feature_importance(xgc,sample_df.head(10),features)
list(df_i['feature'].values)
list(df_i.head(50)['feature'].values)

['ip_app_os_var_hour',
 'ip_channel_var_hour',
 'ip_app_os_count_is_attributed',
 'ip',
 'ip_var_hour',
 'ip_app_channel_mean_hour',
 'ip_mean_hour',
 'ip_app_channel_var_hour',
 'ip_device_os_nunique_app',
 'ip_app_hour_count_is_attributed',
 'ip_device_mean_is_attributed',
 'ip_mean_is_attributed',
 'ip_app_channel_count_is_attributed',
 'app_channel_os_mean_is_attributed',
 'os_hour_mean_is_attributed',
 'app_os_hour_mean_is_attributed',
 'app_channel_hour_mean_is_attributed',
 'ip_app_count_is_attributed',
 'ip_app_nunique_os',
 'ip_device_hour_count_is_attributed',
 'os_channel_mean_is_attributed',
 'os_channel_hour_mean_is_attributed',
 'ip_nunique_app',
 'app_device_os_mean_is_attributed',
 'os_hour_count_is_attributed',
 'ip_hour_count_is_attributed',
 'app_hour_mean_is_attributed',
 'ip_count_is_attributed',
 'app_channel_mean_is_attributed',
 'channel_app_device_hour_nunique_ip',
 'ip_nunique_channel',
 'device_os_mean_hour',
 'ip_nunique_device',
 'device_channel_hour_mean_i

['ip_app_os_var_hour',
 'ip_channel_var_hour',
 'ip_app_os_count_is_attributed',
 'ip',
 'ip_var_hour',
 'ip_app_channel_mean_hour',
 'ip_mean_hour',
 'ip_app_channel_var_hour',
 'ip_device_os_nunique_app',
 'ip_app_hour_count_is_attributed',
 'ip_device_mean_is_attributed',
 'ip_mean_is_attributed',
 'ip_app_channel_count_is_attributed',
 'app_channel_os_mean_is_attributed',
 'os_hour_mean_is_attributed',
 'app_os_hour_mean_is_attributed',
 'app_channel_hour_mean_is_attributed',
 'ip_app_count_is_attributed',
 'ip_app_nunique_os',
 'ip_device_hour_count_is_attributed',
 'os_channel_mean_is_attributed',
 'os_channel_hour_mean_is_attributed',
 'ip_nunique_app',
 'app_device_os_mean_is_attributed',
 'os_hour_count_is_attributed',
 'ip_hour_count_is_attributed',
 'app_hour_mean_is_attributed',
 'ip_count_is_attributed',
 'app_channel_mean_is_attributed',
 'channel_app_device_hour_nunique_ip',
 'ip_nunique_channel',
 'device_os_mean_hour',
 'ip_nunique_device',
 'device_channel_hour_mean_i

In [9]:
df_train = df_train.sample(frac=0.4)
X = df_train[features]
y = df_train[output_column]

df_train = None
del df_train
df_cv1=None
del df_cv1
X.head();
gc.collect()

df_train = None
del df_train
df_cv1=None
del df_cv1
X.head();
gc.collect()

NameError: name 'df_train' is not defined

In [13]:
def check_auc(X,y):
    y_scores = xgc.predict_proba(X)[:,1]
    return roc_auc_score(y, y_scores)




gc.collect()


def conf_mat(X,y):
    y_scores = xgc.predict(X)
    return model_utils.confusion_matrix_frame(y,y_scores)

232

In [ ]:
df_train = None
del df_train
df_cv1=None
del df_cv1

gc.collect()



print("Train Metrics\n")
check_auc(X,y)
conf_mat(X,y)

X=None
y=None
del X
del y
gc.collect()

In [25]:
gc.collect()
df_cv1 = read_cv()
df_cv1 = df_cv1.sample(frac=0.4)
X_test = df_cv1[features]
y_test = df_cv1[output_column]
df_cv1 = None
del df_cv1
gc.collect()

0

22

In [26]:
X_test.head();
gc.collect()

,app_os_hour_mean_is_attributed,device_hour_count_channel,ip_mean_is_attributed,hour_nunique_app,hour,app_mean_is_attributed,app_device_var_hour,device_nunique_os,app_hour_mean_is_attributed,device_channel_mean_is_attributed,channel_nunique_ip,hour_nunique_device,app_os_mean_is_attributed,ip_app_mean_is_attributed,app_nunique_os,app_AvgViewPerDistinct_ip,ip_device_mean_is_attributed,os_count_app,ip_channel_mean_is_attributed,os_app_nunique_device,ip_channel_hour_mean_is_attributed,ip_mean_hour,os_channel_mean_is_attributed,device_nunique_hour,os_channel_hour_mean_is_attributed,app_hour_count_channel,app_channel_hour_mean_is_attributed,ip_os_hour_mean_is_attributed,device,app_nunique_device,os_hour_count_channel,ip_app_os_var_hour,ip_channel_var_hour,ip_app_channel_mean_hour,device_mean_is_attributed,ip_hour_count_channel,os_nunique_app,app_nunique_ip,ip_app_hour_count_channel,app_device_os_mean_is_attributed,hour_nunique_ip,ip_app_count_channel,app_device_mean_is_attributed,ip_app_hour_mean_is_attributed,channel_nunique_os,app_count_channel,app_device_os_nunique_ip,app,hour_nunique_channel,app_channel_os_mean_is_attributed,channel,channel_hour_mean_is_attributed,ip_hour_mean_is_attributed,ip_app_os_count_channel,device_nunique_channel,os_hour_mean_is_attributed,ip_nunique_device,ip_var_hour,os,ip_os_channel_mean_is_attributed,device_channel_hour_mean_is_attributed,hour_mean_is_attributed,app_count_os,ip_app_device_mean_is_attributed,device_var_hour,device_count_os,channel_count_app,os_app_nunique_ip,app_channel_mean_hour,channel_nunique_app,ip_nunique_channel,ip_nunique_app,ip_count_channel,os_mean_is_attributed,device_os_mean_hour,ip_app_channel_mean_is_attributed,app_nunique_channel,ip,hour_nunique_os,channel_mean_is_attributed,device_hour_mean_is_attributed,device_count_app,app_channel_mean_is_attributed,ip_device_hour_mean_is_attributed,device_os_var_hour,device_nunique_app,ip_app_nunique_os,ip_device_os_nunique_app,device_nunique_ip,device_count_channel,device_app_mean_hour
129421594,0.000394,3556828,0.002621,322,9,0.000519,42.080893,537,0.000460,0.001118,8602,869,0.000746,0.000718,191,56.885484,0.003743,1221627,0.001041,3,0.0,10.772033,0.000248,24,0.000000,271539,0.000453,NaN,1,6,58966,29.562500,32.807492,11.655136,0.004935,254,135,117870,37,0.001169,91964,1470,0.000719,0.0,167,6705092,27699,15,174,0.000000,3,0.000904,0.003158,16,189,0.006499,13,36.305864,37,0.0,0.001097,0.006438,6705092,0.001392,38.255844,73278711,368423,28104,11.225221,10,114,53,7795,0.006171,9.405262,0.001562,31,102174,244,0.000831,0.005813,73278711,0.000156,0.006243,38.135178,350,58,15,217490,73278711,9.893285
86185379,0.000531,4525846,0.002005,328,4,0.001235,36.918485,537,0.001390,0.001285,87015,817,0.001021,0.000000,186,55.802444,0.001855,2211621,0.000000,3,0.0,8.145105,0.000737,24,0.001261,381538,0.001689,NaN,1,6,136193,63.000000,43.335714,7.407407,0.004935,86,169,118640,4,0.001532,96201,114,0.001711,NaN,143,6620402,39201,18,179,0.001272,121,0.001154,0.000000,3,189,0.003380,9,38.895812,10,NaN,0.001703,0.006209,6620402,0.000000,38.255844,73278711,1971482,39355,10.765039,2,96,37,1144,0.002722,9.147356,0.000000,18,812,231,0.000967,0.006272,73278711,0.001322,0.000000,37.683192,350,22,11,217490,73278711,9.422918
27889723,0.000292,3885804,0.000000,352,10,0.000603,39.977041,537,0.000416,0.001310,75381,912,0.000715,0.000000,224,37.811832,0.000000,16720972,0.000000,3,NaN,10.305060,0.001283,24,0.001489,222671,0.000595,0.0,1,6,890245,12.622222,28.000000,15.000000,0.004935,48,235,111411,4,0.001082,93595,40,0.000817,NaN,135,4212654,69809,14,175,0.000723,439,0.001097,0.000000,10,189,0.003954,6,32.850168,13,NaN,0.001654,0.005765,4212654,0.000000,38.255844,73278711,1219469,70347,9.150168,3,87,33,672,0.003984,9.253499,NaN,39,100759,253,0.000984,0.005109,73278711,0.000633,0.000000,38.623718,350,11,20,217490,73278711,9.047957
125239289,0.009573,3943909,0.001813,330,3,0.002824,38.732324,537,0.003937,0.002210,73103,757,0.006149,0.001255,216,56.334671,0.001582,3115835,

0

In [27]:






gc.collect()

gc.collect()

print("Test Metrics\n")
check_auc(X_test,y_test)
conf_mat(X_test,y_test)

0

0

Test Metrics



0.9831393785549921

Predicted,0,1,Actual Counts,Recall %
Actual,,,,
0,12395757,125,12395882.0,99.0
1,37992,5540,43532.0,12.0
Predicted Counts,12433749,5665,12439414.0,NaN
Precision %,99,97,NaN,NaN


In [15]:
gc.collect()
y_test = None
X_test = None
del X_test
del y_test
gc.collect()

0

26

In [6]:
gc.collect()
df_cv1 = None
del df_cv1


gc.collect()
df_test = pd.read_hdf("data/test_2.h5","table")

41

0

In [7]:
gc.collect()
results = df_test[["click_id"]]
df_test = df_test[features]
gc.collect()
results['is_attributed'] = xgc.predict_proba(df_test)[:,1]

0

21

In [8]:
gc.collect()
df_test = None
del df_test
gc.collect()

gc.collect()
gc.collect()

0

0

0

0

In [9]:
results.to_csv("results/results-8.csv",index=False,columns=results.columns)

In [10]:

%cd results

!kaggle competitions submit -c talkingdata-adtracking-fraud-detection -f results-8.csv -m "Xgb with extra features"
%cd ..

/home/ahemf/data-science/kaggle/talkingdata-adtracking-fraud-detection/results
Successfully submitted to TalkingData AdTracking Fraud Detection Challenge/home/ahemf/data-science/kaggle/talkingdata-adtracking-fraud-detection


In [21]:
import pickle
pickle.dump(xgc, open("xgc.pickle.dat", "wb"))

In [23]:
xgc = pickle.load(open("xgc.pickle.dat", "rb"))

In [28]:
gc.collect()

7886

In [30]:
booster = xgc.get_booster()

In [31]:
booster.save_model("xgc.model")

In [32]:
booster.load_model("xgc.model")